In [ ]:
!pip install keras-tuner

In [76]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from keras_tuner import RandomSearch
import numpy as np
from sklearn.metrics import confusion_matrix,accuracy_score,plot_confusion_matrix
import pandas as pd

In [39]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [40]:
len(train_images), len(test_images)

(60000, 10000)

In [41]:
train_images[0].shape

(28, 28)

In [42]:
# normalization
train_images = train_images/255.0
test_images = test_images/255.0

In [43]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [44]:
def build_model(hp):
    model = keras.Sequential()
    model.add(
        layers.Conv2D(
            filters=hp.Int("conv_1_filter", min_value=32, max_value=128, step=16),
            kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
            activation="relu",
            input_shape=(28,28,1)
        )
    )

    model.add(
            layers.Conv2D(
                filters=hp.Int("conv_2_filter", min_value=32, max_value=64, step=16),
                kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
                activation="relu",
            )
        )
    
    model.add(layers.Flatten())

    model.add(
        layers.Dense(
            units=hp.Int("dense_1_units", min_value=32, max_value=128, step=32),
            activation="relu",
        )
    )

    model.add(layers.Dense(10, activation="softmax"))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])
        ),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

In [45]:
tuner = RandomSearch(
    build_model,
    objective="val_accuracy",
    max_trials=3,
    executions_per_trial=2,
    overwrite=True,
    directory="my_dir",
    project_name="CNN model",
)

In [46]:
tuner.search_space_summary()

Search space summary
Default search space size: 6
conv_1_filter (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 16, 'sampling': None}
conv_1_kernel (Choice)
{'default': 3, 'conditions': [], 'values': [3, 5], 'ordered': True}
conv_2_filter (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 64, 'step': 16, 'sampling': None}
conv_2_kernel (Choice)
{'default': 3, 'conditions': [], 'values': [3, 5], 'ordered': True}
dense_1_units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [47]:
tuner.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 3 Complete [00h 02m 05s]
val_accuracy: 0.8768333494663239

Best val_accuracy So Far: 0.8885833323001862
Total elapsed time: 00h 04m 53s
INFO:tensorflow:Oracle triggered exit


In [48]:
tuner.results_summary()

Results summary
Results in my_dir/CNN model
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
conv_1_filter: 64
conv_1_kernel: 3
conv_2_filter: 48
conv_2_kernel: 5
dense_1_units: 64
learning_rate: 0.0001
Score: 0.8885833323001862
Trial summary
Hyperparameters:
conv_1_filter: 64
conv_1_kernel: 3
conv_2_filter: 48
conv_2_kernel: 5
dense_1_units: 32
learning_rate: 0.0001
Score: 0.8846666812896729
Trial summary
Hyperparameters:
conv_1_filter: 64
conv_1_kernel: 3
conv_2_filter: 48
conv_2_kernel: 5
dense_1_units: 96
learning_rate: 0.01
Score: 0.8768333494663239


In [ ]:
model = tuner.get_best_models(num_models=1)[0]

In [56]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 48)        76848     
_________________________________________________________________
flatten (Flatten)            (None, 23232)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                1486912   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                650       
Total params: 1,565,050
Trainable params: 1,565,050
Non-trainable params: 0
_________________________________________________________________


In [57]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 14s 8ms/step - loss: 0.2651 - accuracy: 0.9042 - val_loss: 0.2761 - val_accuracy: 0.8987
Epoch 5/10
1688/1688 [==============================] - 13s 8ms/step - loss: 0.2379 - accuracy: 0.9131 - val_loss: 0.2616 - val_accuracy: 0.9065
Epoch 6/10
1688/1688 [==============================] - 13s 8ms/step - loss: 0.2147 - accuracy: 0.9221 - val_loss: 0.2480 - val_accuracy: 0.9118
Epoch 7/10
1688/1688 [==============================] - 13s 8ms/step - loss: 0.1936 - accuracy: 0.9293 - val_loss: 0.2455 - val_accuracy: 0.9098
Epoch 8/10
1688/1688 [==============================] - 13s 8ms/step - loss: 0.1743 - accuracy: 0.9373 - val_loss: 0.2439 - val_accuracy: 0.9148
Epoch 9/10
1688/1688 [==============================] - 13s 8ms/step - loss: 0.1575 - accuracy: 0.9427 - val_loss: 0.2430 - val_accuracy: 0.9150
Epoch 10/10
1688/1688 [==============================] - 13s 8ms/step - loss: 0.1412 - accuracy: 0.9490 - val_loss: 0.2488 - val_a

In [58]:
y_pred = model.predict(test_images)

In [71]:
result = np.argmax(y_pred, axis=1)

In [74]:
print(accuracy_score(result,test_labels))

0.9098


In [82]:
pd.DataFrame(confusion_matrix(test_labels,result))

,0,1,2,3,4,5,6,7,8,9
0,862,0,21,9,4,1,95,0,8,0
1,3,974,0,14,4,0,4,0,1,0
2,17,1,839,4,81,0,55,0,3,0
3,18,1,12,902,36,0,27,0,4,0
4,1,0,34,19,893,0,53,0,0,0
5,0,0,0,0,0,993,0,5,0,2
6,98,1,54,19,70,0,751,0,7,0
7,0,0,0,0,0,21,0,971,0,8
8,4,0,3,2,1,2,4,4,980,0
9,1,0,0,0,0,11,0,55,0,933
